Submitting various things for end of grant.

In [1]:
import os
import sys
import requests
import pandas
import paramiko
import json
from IPython import display

In [2]:
from curation_common import *
from htsworkflow.submission.encoded import DCCValidator

In [3]:
PANDAS_ODF = os.path.expanduser('~/src/odf_pandas')
if PANDAS_ODF not in sys.path:
    sys.path.append(PANDAS_ODF)
    from pandasodf import ODFReader

In [4]:
from htsworkflow.submission.encoded import Document
from htsworkflow.submission.aws_submission import run_aws_cp

In [5]:
# live server & control file
server = ENCODED('www.encodeproject.org')
spreadsheet_name = os.path.expanduser('~diane/woldlab/ENCODE/C1-mouse-forlimb-submission-201804.ods')

# test server & datafile
#server = ENCODED('test.encodedcc.org')
#spreadsheet_name = os.path.expanduser('~diane/woldlab/ENCODE/C1-mouse-forlimb-submission-201804-testserver.ods')

server.load_netrc()
validator = DCCValidator(server)

In [6]:
award = 'UM1HG009443'

# Submit Documents

Example Document submission

In [7]:
#atac_uuid = '0fc44318-b802-474e-8199-f3b6d708eb6f'
#atac = Document(os.path.expanduser('~/proj/encode3-curation/Wold_Lab_ATAC_Seq_protocol_December_2016.pdf'),
#                'general protocol',
#                'ATAC-Seq experiment protocol for Wold lab',
#                )
#body = atac.create_if_needed(server, atac_uuid)
#print(body['@id'])

# Submit Annotations

In [8]:
#sheet = gcat.get_file(spreadsheet_name, fmt='pandas_excel')
#annotations = sheet.parse('Annotations', header=0)
#created = server.post_sheet('/annotations/', annotations, verbose=True, dry_run=True)
#print(len(created))

In [9]:
#if created:
#    annotations.to_excel('/tmp/annotations.xlsx', index=False)

# Register Biosamples

In [18]:
book = ODFReader(spreadsheet_name)
biosample = book.parse('Biosamples', header=0)
created = server.post_sheet('/biosamples/', biosample, 
                            verbose=True, 
                            dry_run=True, 
                            validator=validator)
print(len(created))

{'biosample_term_id': 'UBERON:0002102', 'biosample_term_name': 'forelimb', 'description': 'C1 e10.5 mouse forelimb run15', 'aliases': ['barbara-wold:C1_e10.5_AM_run_mouse_forelimb_run15_January13_2018'], 'biosample_type': 'single cell', 'date_obtained': '2018-01-13', 'organism': '/organisms/mouse', 'source': '/sources/jackson-labs/', 'starting_amount': 18, 'starting_amount_units': 'cells', 'model_organism_age': '10.5', 'model_organism_age_units': 'day', 'mouse_life_stage': 'embryonic', 'donor': '/mouse-donors/ENCDO956IXV/', 'lab': 'barbara-wold', 'award': 'UM1HG009443'} biosample_accession
{'biosample_term_id': 'UBERON:0002102', 'biosample_term_name': 'forelimb', 'description': 'C1 e11.0 mouse forelimb run16', 'aliases': ['barbara-wold:C1_e11.0_PM_run_mouse_forelimb_run16_January13_2018'], 'biosample_type': 'single cell', 'date_obtained': '2018-01-13', 'organism': '/organisms/mouse', 'source': '/sources/jackson-labs/', 'starting_amount': 14, 'starting_amount_units': 'cells', 'model_org

Reponse {'@graph': [{'description': 'C1 e12.0 mouse forelimb run11', 'dbxrefs': [], 'biosample_synonyms': ['Entire upper limb', 'upper limb', 'fore limb', 'foreleg', 'pectoral flipper', 'free upper limb', 'pectoral limb', 'superior member', 'free part of upper limb', 'upper extremity', 'membrum superius', 'anteriormost limb', 'forelimb'], 'references': [], 'life_stage': 'embryonic', 'status': 'in progress', 'internal_tags': [], 'age_display': '12.0 day', 'characterizations': [], 'date_obtained': '2017-12-12', 'award': '/awards/UM1HG009443/', 'parent_of': [], 'model_organism_age': '12.0', 'source': '/sources/jackson-labs/', 'donor': '/mouse-donors/ENCDO956IXV/', 'model_organism_age_units': 'day', 'system_slims': [], 'applied_modifications': [], 'developmental_slims': [], 'aliases': ['barbara-wold:C1_e12.0_mouse_forelimb_run11_December12_2017'], 'date_created': '2018-04-30T17:00:08.116250+00:00', 'starting_amount_units': 'cells', 'age_units': 'day', 'organism': '/organisms/mouse/', 'subm

In [19]:
if created:
    biosample.to_excel('/dev/shm/biosamples.xlsx', index=False)

# Register Libraries

In [27]:
print(spreadsheet_name)
book = ODFReader(spreadsheet_name)
libraries = book.parse('Libraries', header=0)
created = server.post_sheet('/libraries/', 
                            libraries, 
                            verbose=True, 
                            dry_run=True, 
                            validator=validator)
print(len(created))

/home/diane/woldlab/ENCODE/C1-mouse-forlimb-submission-201804.ods
{'aliases': ['barbara-wold:19912_G10', 'barbara-wold:19912_G11', 'barbara-wold:19912_G12', 'barbara-wold:19912_G7', 'barbara-wold:19912_G8', 'barbara-wold:19912_G9', 'barbara-wold:19913_H1', 'barbara-wold:19913_H10', 'barbara-wold:19913_H11', 'barbara-wold:19913_H12', 'barbara-wold:19913_H2', 'barbara-wold:19913_H3', 'barbara-wold:19913_H4', 'barbara-wold:19913_H5', 'barbara-wold:19913_H6', 'barbara-wold:19913_H7', 'barbara-wold:19913_H8', 'barbara-wold:19913_H9'], 'nucleic_acid_term_name': 'polyadenylated mRNA', 'biosample': 'barbara-wold:C1_e10.5_AM_run_mouse_forelimb_run15_January13_2018', 'spikeins_used': ['barbara-wold:profile_C1_3'], 'documents': ['barbara-wold:smarter-ulta-low-pdf'], 'lab': 'barbara-wold', 'award': 'UM1HG009443', 'barcode_details': [{'barcode': 'GTAGAGGACGTCTAAT', 'plate_id': '19912', 'plate_location': 'G10'}, {'barcode': 'GCTCATGACGTCTAAT', 'plate_id': '19912', 'plate_location': 'G11'}, {'barcode

Reponse {'@graph': [{'spikeins_used': ['/references/ENCSR535LMC/'], '@type': ['Library', 'Item'], 'uuid': '7b7dc8d2-784b-4d04-a995-6bd7b9592b38', 'barcode_details': [{'barcode': 'ACTCGCTATCGACTAG', 'plate_location': 'A1', 'plate_id': '19914'}, {'barcode': 'CGATCAGTTCGACTAG', 'plate_location': 'A10', 'plate_id': '19914'}, {'barcode': 'TGCAGCTATCGACTAG', 'plate_location': 'A11', 'plate_id': '19914'}, {'barcode': 'TCGACGTCTCGACTNG', 'plate_location': 'A12', 'plate_id': '19914'}, {'barcode': 'GGAGCTACTCGACTAG', 'plate_location': 'A2', 'plate_id': '19914'}, {'barcode': 'GCGTAGTATCGACTNG', 'plate_location': 'A3', 'plate_id': '19914'}, {'barcode': 'CGGAGCCTTCGACTAG', 'plate_location': 'A4', 'plate_id': '19914'}, {'barcode': 'TACGCTGCTCGACTAG', 'plate_location': 'A5', 'plate_id': '19914'}, {'barcode': 'ATGCGCAGTCGACTAG', 'plate_location': 'A6', 'plate_id': '19914'}, {'barcode': 'TAGCGCTCTCGACTAG', 'plate_location': 'A7', 'plate_id': '19914'}, {'barcode': 'ACTGAGCGTCGACTAG', 'plate_location': '

Reponse {'@graph': [{'treatments': [], 'documents': ['/documents/49fd5b3b-9878-4ec8-92bf-ef6dce9116e3/'], '@type': ['Library', 'Item'], 'nucleic_acid_term_id': 'SO:0000871', 'strand_specificity': False, 'nucleic_acid_term_name': 'polyadenylated mRNA', 'biosample': '/biosamples/ENCBS875OEX/', '@id': '/libraries/ENCLB685FNO/', 'status': 'in progress', 'aliases': ['barbara-wold:20026_A1', 'barbara-wold:20026_A10', 'barbara-wold:20026_A11', 'barbara-wold:20026_A12', 'barbara-wold:20026_A2', 'barbara-wold:20026_A3', 'barbara-wold:20026_A4', 'barbara-wold:20026_A5', 'barbara-wold:20026_A6', 'barbara-wold:20026_A7', 'barbara-wold:20026_A8', 'barbara-wold:20026_A9', 'barbara-wold:20027_B1', 'barbara-wold:20027_B10', 'barbara-wold:20027_B11', 'barbara-wold:20027_B12', 'barbara-wold:20027_B2', 'barbara-wold:20027_B3', 'barbara-wold:20027_B4', 'barbara-wold:20027_B5', 'barbara-wold:20027_B6', 'barbara-wold:20027_B7', 'barbara-wold:20027_B8', 'barbara-wold:20027_B9', 'barbara-wold:20028_C1', 'barb

Reponse {'@type': ['result'], '@graph': [{'@id': '/libraries/ENCLB812YXV/', 'date_created': '2018-04-30T17:06:19.800243+00:00', 'treatments': [], 'status': 'in progress', 'lab': '/labs/barbara-wold/', 'accession': 'ENCLB812YXV', 'nucleic_acid_term_name': 'polyadenylated mRNA', 'biosample': '/biosamples/ENCBS621IQO/', 'documents': ['/documents/49fd5b3b-9878-4ec8-92bf-ef6dce9116e3/'], '@type': ['Library', 'Item'], 'uuid': 'c0ed430e-f0dd-4e30-9a38-581c7b484094', 'nucleic_acid_term_id': 'SO:0000871', 'strand_specificity': False, 'schema_version': '8', 'aliases': ['barbara-wold:20039_A1', 'barbara-wold:20039_A10', 'barbara-wold:20039_A11', 'barbara-wold:20039_A12', 'barbara-wold:20039_A2', 'barbara-wold:20039_A3', 'barbara-wold:20039_A4', 'barbara-wold:20039_A5', 'barbara-wold:20039_A6', 'barbara-wold:20039_A7', 'barbara-wold:20039_A8', 'barbara-wold:20039_A9', 'barbara-wold:20040_B1', 'barbara-wold:20040_B10', 'barbara-wold:20040_B11', 'barbara-wold:20040_B12', 'barbara-wold:20040_B2', 'ba

Reponse {'status': 'success', '@graph': [{'date_created': '2018-04-30T17:06:20.163956+00:00', 'aliases': ['barbara-wold:20044_A1', 'barbara-wold:20044_A10', 'barbara-wold:20044_A11', 'barbara-wold:20044_A12', 'barbara-wold:20044_A2', 'barbara-wold:20044_A3', 'barbara-wold:20044_A4', 'barbara-wold:20044_A5', 'barbara-wold:20044_A6', 'barbara-wold:20044_A9', 'barbara-wold:20045_B1', 'barbara-wold:20045_B10', 'barbara-wold:20045_B11', 'barbara-wold:20045_B12', 'barbara-wold:20045_B2', 'barbara-wold:20045_B3', 'barbara-wold:20045_B4', 'barbara-wold:20045_B5', 'barbara-wold:20045_B6', 'barbara-wold:20045_B9', 'barbara-wold:20046_C1', 'barbara-wold:20046_C10', 'barbara-wold:20046_C11', 'barbara-wold:20046_C12', 'barbara-wold:20046_C2', 'barbara-wold:20046_C3', 'barbara-wold:20046_C4', 'barbara-wold:20046_C5', 'barbara-wold:20046_C6', 'barbara-wold:20046_C9', 'barbara-wold:20047_D1', 'barbara-wold:20047_D10', 'barbara-wold:20047_D11', 'barbara-wold:20047_D12', 'barbara-wold:20047_D2', 'barbar

In [28]:
if created:
    libraries.to_excel('/dev/shm/libraries.xlsx', index=False)

In [24]:
len(created)

0

# Register Experiments

In [29]:
book = ODFReader(spreadsheet_name)
experiments = book.parse('Experiments', header=0)
created = server.post_sheet('/experiments/', 
                            experiments, 
                            verbose=True, 
                            dry_run=True, 
                            validator=validator)
print(len(created))

{'description': 'C1 e10.5 mouse forelimb run 15', 'assay_term_name': 'single cell isolation followed by RNA-seq', 'biosample_type': 'single cell', 'biosample_term_name': 'forelimb', 'biosample_term_id': 'UBERON:0002102', 'aliases': ['barbara-wold:c1_e10.5_mouse_limb_run15'], 'lab': 'barbara-wold', 'award': 'UM1HG009443'} experiment_accession
{'description': 'C1 e11.0 mouse forelimb run 16', 'assay_term_name': 'single cell isolation followed by RNA-seq', 'biosample_type': 'single cell', 'biosample_term_name': 'forelimb', 'biosample_term_id': 'UBERON:0002102', 'aliases': ['barbara-wold:c1_e11.0_mouse_limb_run16'], 'lab': 'barbara-wold', 'award': 'UM1HG009443'} experiment_accession
{'description': 'C1 e11.5 mouse forelimb run 10', 'assay_term_name': 'single cell isolation followed by RNA-seq', 'biosample_type': 'single cell', 'biosample_term_name': 'forelimb', 'biosample_term_id': 'UBERON:0002102', 'aliases': ['barbara-wold:c1_e11.5_mouse_limb_run10'], 'lab': 'barbara-wold', 'award': 'UM1

Reponse {'@graph': [{'uuid': '119a192a-abd3-4312-b006-68278da1df91', 'superseded_by': [], 'date_created': '2018-04-30T17:07:10.713005+00:00', 'revoked_files': [], 'accession': 'ENCSR938RJZ', 'related_files': [], 'lab': '/labs/barbara-wold/', 'references': [], 'system_slims': [], 'documents': [], 'assay_term_name': 'single cell isolation followed by RNA-seq', 'assay_term_id': 'NTR:0003082', 'assay_title': 'single cell RNA-seq', 'objective_slims': [], '@type': ['Experiment', 'Dataset', 'Item'], 'type_slims': [], 'category_slims': [], 'description': 'C1 e14.0 mouse forelimb run 13', 'submitted_by': '/users/bc5b62f7-ce28-4a1e-b6b3-81c9c5a86d7a/', '@id': '/experiments/ENCSR938RJZ/', 'possible_controls': [], 'biosample_type': 'single cell', 'replication_type': 'unreplicated', 'alternate_accessions': [], 'status': 'in progress', 'internal_status': 'unreviewed', 'supersedes': [], 'award': '/awards/UM1HG009443/', 'original_files': [], 'dbxrefs': [], 'replicates': [], 'contributing_files': [], '

In [30]:
if created:
    experiments.to_excel('/dev/shm/experiments.xlsx', index=False)

# Register Replicates

In [31]:
book = ODFReader(spreadsheet_name)
replicates = book.parse('Replicates', header=0)
created = server.post_sheet('/replicates/', 
                            replicates, 
                            verbose=True, 
                            dry_run=True, 
                            validator=validator)
print(len(created))

{'experiment': 'barbara-wold:c1_e10.5_mouse_limb_run15', 'biological_replicate_number': 1, 'technical_replicate_number': 1, 'library': 'barbara-wold:19912_G10'} uuid
{'experiment': 'barbara-wold:c1_e11.0_mouse_limb_run16', 'biological_replicate_number': 1, 'technical_replicate_number': 1, 'library': 'barbara-wold:19914_A1'} uuid
{'experiment': 'barbara-wold:c1_e11.5_mouse_limb_run10', 'biological_replicate_number': 1, 'technical_replicate_number': 1, 'library': 'barbara-wold:19906_A1'} uuid
{'experiment': 'barbara-wold:c1_e12.0_mouse_limb_run11', 'biological_replicate_number': 1, 'technical_replicate_number': 1, 'library': 'barbara-wold:20026_A1'} uuid
{'experiment': 'barbara-wold:c1_e13.0_mouse_limb_run12', 'biological_replicate_number': 1, 'technical_replicate_number': 1, 'library': 'barbara-wold:20033_A1'} uuid
{'experiment': 'barbara-wold:c1_e14.0_mouse_limb_run13', 'biological_replicate_number': 1, 'technical_replicate_number': 1, 'library': 'barbara-wold:20039_A1'} uuid
{'experim

In [32]:
if created:
    replicates.to_excel('/dev/shm/replicates.xlsx', index=False)

# Make manifest

In [75]:
def make_manifest(spreadsheet_name, submission_root, map_name):
    submission_name = 'C1_mouse_limb_submission_201804'
    book = ODFReader(spreadsheet_name)
    libraries = book.parse('Libraries')
    replicates = book.parse('Replicates')
    
    pongo = paramiko.SSHClient()
    pongo.set_missing_host_key_policy(paramiko.AutoAddPolicy())
    pongo.connect('pongo.caltech.edu', username='diane')
    sftp = pongo.open_sftp()
    sftp.chdir(submission_root)
    files = 0
    for line in sftp.open(map_name):
        # skip comments
        line = line.strip()
        if len(line) == 0 or line.startswith('#'):
            continue

        library_id, dirname = line.split()
        alias = 'barbara-wold:{}'.format(library_id)
        library_row = libraries[[alias in row for row in list(libraries['aliases:array'])]]
        replicate_row = replicates[[alias in row for row in list(libraries['aliases:array'])]]
        for filename in sftp.listdir(dirname):
            if filename.endswith('fastq.gz'):
                relative_path = os.path.join(dirname, filename)
                metadata = {
                    'dataset': list(replicate_row['experiment'])[0],
                    'submitted_file_name': relative_path,
                    'replicate': list(replicate_row['uuid'])[0],
                    'lab': list(library_row['lab'])[0],
                    'award': list(library_row['award'])[0],
                }
                #display.display_pretty(metadata)
                print('<http://jumpgate.caltech.edu/wiki/SubmissionsLog/{}#{}>'.format(submission_name, dirname))
                print('  encode3:dataset "{}" ;'.format(metadata['dataset']))
                print('  encode3:replicate "{}" ;'.format(metadata['replicate']))
                print('  encode3:lab "{}" ;'.format(metadata['lab']))
                print('  encode3:award "{}" .'.format(metadata['award']))

                print('<file:///woldlab/loxcyc/home/diane/{}/{}>'.format(submission_root, relative_path))
                print('  encode3:output_type "reads" ;')
                print('  encode3:read_length 100 ; ')
                print('  encode3:file_format "fastq" .')
                files += 1

    print('generated {} records'.format(files))


In [76]:
make_manifest(spreadsheet_name, 'proj/C1_mouse_limb_combined', 'libs.txt')

<http://jumpgate.caltech.edu/wiki/SubmissionsLog/C1_mouse_limb_submission_201804#ENCLB142PSQ>
  encode3:dataset "barbara-wold:c1_e10.5_mouse_limb_run15" ;
  encode3:replicate "43cee703-3165-4191-97c2-cfa014c9687a" ;
  encode3:lab "barbara-wold" ;
  encode3:award "UM1HG009443" .
<file:///woldlab/loxcyc/home/diane/proj/C1_mouse_limb_combined/ENCLB142PSQ/C1_mouse_e10.5_forelimb_run15_January13AM_2018.fastq.gz>
  encode3:output_type "reads" ;
  encode3:read_length 100 ; 
  encode3:file_format "fastq" .
<http://jumpgate.caltech.edu/wiki/SubmissionsLog/C1_mouse_limb_submission_201804#ENCLB137HIJ>
  encode3:dataset "barbara-wold:c1_e11.0_mouse_limb_run16" ;
  encode3:replicate "5e83d1a3-9c3e-4dfb-9c9e-956170c66ffe" ;
  encode3:lab "barbara-wold" ;
  encode3:award "UM1HG009443" .
<file:///woldlab/loxcyc/home/diane/proj/C1_mouse_limb_combined/ENCLB137HIJ/C1_mouse_e11.0_forelimb_run16_January13PM_2018.fastq.gz>
  encode3:output_type "reads" ;
  encode3:read_length 100 ; 
  encode3:file_format "f